# Preprocess Pseudo-tubes

Applies normalization, log1p, and HVG filtering to the raw pseudo-tubes built by `pseudo_tubes_creation.ipynb`.

**Strategy (memory-efficient):**
1. Load one pseudo-tube per cytokine (rotating across donors) from the existing files and concatenate them — ~91 cytokines × ~450 cells ≈ 40k cells, trivially small.
2. Normalize + log1p + `highly_variable_genes` on this representative sample. Save `hvg_list.json`.
3. Loop through all tubes from the manifest one at a time:
   - Rename `pseudotube_N.h5ad` → `pseudotube_N_raw.h5ad` (keeps original intact)
   - Normalize → log1p → filter to HVGs
   - Save preprocessed as `pseudotube_N.h5ad`

**Resume-safe:** skips tubes where both `pseudotube_N_raw.h5ad` and `pseudotube_N.h5ad` already exist.

In [1]:
import scanpy as sc
import numpy as np
import os
import json
from collections import defaultdict
from tqdm import tqdm

In [2]:
BASE_PATH     = "/cs/labs/mornitzan/yam.arieli/datasets/Oesinghaus_pseudotubes"
MANIFEST_PATH = os.path.join(BASE_PATH, "manifest.json")
HVG_PATH      = os.path.join(BASE_PATH, "hvg_list.json")

N_TOP_GENES      = 4000  # HVGs to keep (CLAUDE.md §2: treat as hyperparameter)
NORMALIZE_TARGET = 1e4

## 1. Load manifest

In [3]:
with open(MANIFEST_PATH) as f:
    manifest = json.load(f)

print(f"Manifest entries : {len(manifest)}")
cytokines = sorted({e['cytokine'] for e in manifest})
donors    = sorted({e['donor']    for e in manifest})
print(f"Cytokines        : {len(cytokines)}")
print(f"Donors           : {len(donors)}")

Manifest entries : 10920
Cytokines        : 91
Donors           : 12


## 2. Select HVGs from one tube per cytokine

For each cytokine we pick one tube, rotating across donors so the sample spans the full donor diversity.
Result: ~91 tubes × ~450 cells ≈ 40k cells — enough for stable HVG estimation, trivial memory.

In [4]:
if os.path.exists(HVG_PATH):
    print("hvg_list.json already exists — loading from disk.")
    with open(HVG_PATH) as f:
        hvg_list = json.load(f)
    print(f"HVGs loaded: {len(hvg_list)}")
else:
    # Group manifest entries by cytokine, sorted by donor for reproducibility
    cyt_to_entries = defaultdict(list)
    for entry in manifest:
        cyt_to_entries[entry['cytokine']].append(entry)

    # Pick one tube per cytokine — rotate donor index so the sample
    # spans all 12 donors rather than always using Donor1
    sample_paths = []
    for i, cyt in enumerate(sorted(cyt_to_entries)):
        entries = sorted(cyt_to_entries[cyt], key=lambda e: e['donor'])
        # tube_idx=0 from rotating donor
        candidates = [e for e in entries if e['tube_idx'] == 0]
        picked = candidates[i % len(candidates)]
        sample_paths.append(picked['path'])

    print(f"Loading {len(sample_paths)} tubes for HVG estimation...")
    adatas = [sc.read_h5ad(p) for p in tqdm(sample_paths, desc="Reading tubes")]
    adata_hvg = sc.concat(adatas, join='outer', label='cytokine',
                          keys=[os.path.basename(os.path.dirname(p)) for p in sample_paths])
    del adatas
    print(f"Concatenated shape: {adata_hvg.shape}")

    print("Normalizing...")
    sc.pp.normalize_total(adata_hvg, target_sum=NORMALIZE_TARGET)
    sc.pp.log1p(adata_hvg)

    print(f"Selecting top {N_TOP_GENES} HVGs...")
    sc.pp.highly_variable_genes(adata_hvg, n_top_genes=N_TOP_GENES)
    hvg_list = adata_hvg.var_names[adata_hvg.var['highly_variable']].tolist()
    del adata_hvg

    with open(HVG_PATH, 'w') as f:
        json.dump(hvg_list, f, indent=2)
    print(f"Saved {len(hvg_list)} HVGs to {HVG_PATH}")

hvg_list.json already exists — loading from disk.
HVGs loaded: 4000


## 3. Preprocess all tubes (one at a time)

In [5]:
def _is_preprocessed(path):
    """True if the preprocessed file exists and the raw backup also exists."""
    raw_path = path.replace(".h5ad", "_raw.h5ad")
    return os.path.exists(path) and os.path.exists(raw_path)


def preprocess_tube(path, hvg_list, normalize_target):
    """
    Rename the raw tube, preprocess, and save under the original name.

    Steps:
      1. Rename  pseudotube_N.h5ad -> pseudotube_N_raw.h5ad
      2. Load raw, normalize_total, log1p, filter to hvg_list
      3. Save preprocessed as pseudotube_N.h5ad
    """
    raw_path = path.replace(".h5ad", "_raw.h5ad")

    if not os.path.exists(raw_path):
        os.rename(path, raw_path)

    adata = sc.read_h5ad(raw_path)
    sc.pp.normalize_total(adata, target_sum=normalize_target)
    sc.pp.log1p(adata)

    # Guard against sc.concat join='outer' edge cases
    genes_present = [g for g in hvg_list if g in adata.var_names]
    adata = adata[:, genes_present].copy()

    adata.write_h5ad(path, compression='gzip')
    return len(genes_present)

In [6]:
n_skipped = 0
n_done    = 0
n_missing_genes_warn = []

for entry in tqdm(manifest, desc="Preprocessing tubes"):
    path = entry["path"]

    if _is_preprocessed(path):
        n_skipped += 1
        continue

    if not os.path.exists(path) and not os.path.exists(path.replace(".h5ad", "_raw.h5ad")):
        print(f"WARNING: tube missing entirely — {path}")
        continue

    n_genes = preprocess_tube(path, hvg_list, NORMALIZE_TARGET)
    if n_genes < len(hvg_list):
        n_missing_genes_warn.append((path, n_genes))
    n_done += 1

print(f"\nDone.")
print(f"  Preprocessed : {n_done}")
print(f"  Already done : {n_skipped}")
if n_missing_genes_warn:
    print(f"  Tubes with missing HVGs: {len(n_missing_genes_warn)} (see n_missing_genes_warn)")

Preprocessing tubes: 100%|██████████████████████████████████████| 10920/10920 [15:18<00:00, 11.89it/s]


Done.
  Preprocessed : 3638
  Already done : 7282


## 4. Sanity check — spot-check one tube

In [7]:
sample_entry = manifest[0]
sample_path  = sample_entry["path"]
sample_raw   = sample_path.replace(".h5ad", "_raw.h5ad")

adata_pre = sc.read_h5ad(sample_path)
adata_raw = sc.read_h5ad(sample_raw)

print(f"Tube: {sample_entry['donor']} / {sample_entry['cytokine']} / tube {sample_entry['tube_idx']}")
print(f"  Raw shape     : {adata_raw.shape}")
print(f"  Preprocessed  : {adata_pre.shape}")
print(f"  Expected HVGs : {len(hvg_list)}")
print()
print(f"  Raw — min={adata_raw.X.min():.2f}  max={adata_raw.X.max():.2f}  mean={adata_raw.X.mean():.4f}")
print(f"  Pre — min={adata_pre.X.min():.2f}  max={adata_pre.X.max():.2f}  mean={adata_pre.X.mean():.4f}")
print()
assert set(adata_pre.var_names) <= set(hvg_list), "Unexpected genes in preprocessed tube!"
print("Gene check passed: all columns are HVGs.")

Tube: Donor1 / 4-1BBL / tube 0
  Raw shape     : (530, 40352)
  Preprocessed  : (530, 4000)
  Expected HVGs : 4000

  Raw — min=0.00  max=2365.00  mean=0.1475
  Pre — min=0.00  max=6.99  mean=0.0814

Gene check passed: all columns are HVGs.
